In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D

sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df_raw = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older.xlsx', sheet_name='raw')
df_p = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older.xlsx', sheet_name='p<0.05')
df_l = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older.xlsx', sheet_name='log2fc>=1')

In [ ]:
df_old=pd.merge(df_p, df_raw, on='gene_id', how='left')


In [ ]:
df_older=pd.merge(df_l, df_old, on='gene_id', how='left')
df_older

In [ ]:
df_older.to_excel('C:/data/CORBIN/Graber/new/adult_older_1.xlsx')

In [ ]:
df1_raw = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder.xlsx', sheet_name='raw')
df1_p = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder.xlsx', sheet_name='p<0.05')
df1_l = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder.xlsx', sheet_name='log2fc>=1')


In [ ]:
df_eld=pd.merge(df1_p, df1_raw, on='gene_id', how='left')
df_elder=pd.merge(df_eld, df1_l, on='gene_id', how='right')
df_elder.describe()

In [ ]:
df_elder.to_excel('C:/data/CORBIN/Graber/new/adult_elder_1.xlsx')

In [ ]:
def regviz (data, uge):
    x=data['CFAB']
    y=data[uge]
    data_fit = np.polyfit(x, y, 1)
    slope, intercept, r, p_value, std_err = stats.linregress(x,y)
    fig, ax = plt.subplots(figsize=(10,8))
    
    #sns.regplot(x=x, y=y, color='deeppink', ci=0, marker='^')
    data1=data[data['age']==data['age'].value_counts().index[1]]
    data2=data[data['age']==data['age'].value_counts().index[0]]
    ax.scatter(data1['CFAB'], data1[uge], c='green', marker='^', alpha=1, s=400)
    ax.scatter(data2['CFAB'], data2[uge], c='red', marker='^', alpha=1, s=400)
#     ax1 = data1.plot(kind='scatter', x='cfab', y=uge, color='green', alpha=1, figsize=(10, 7), marker='^', s=400)
#     ax1 = data2.plot(kind='scatter', x='cfab', y=uge, color='red', alpha=1, figsize=(10, 7), marker='^', s=400)
    plt.plot(x, data_fit[0] * x + data_fit[1], color='darkblue', linestyle='dotted')
    old_elder0=data['age'].value_counts().index[0]
    old_elder1=data['age'].value_counts().index[1]
    plt.title('CFAB vs. '+uge+'(green:age {}, red:age {})'.format(old_elder1, old_elder0), size=24)
    
    plt.xlabel('CFAB', size=18)
    plt.ylabel('Log2 fold expression', size=18)
    if slope<0:
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/len(y), 'y={:.2f}+{:.2f}*x'.format(data_fit[1], data_fit[0]), color='black', size=15)
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/(len(y)-12), 'R2={:.2f}'.format(r**2), color='black', size=15)
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/(len(y)-15), 'P-value={:.2f}'.format(p_value), color='black', size=15)
    else:
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)-15), 'y={:.2f}+{:.2f}*x'.format(data_fit[1], data_fit[0]), color='black', size=15)
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)-12), 'R2={:.2f}'.format(r**2), color='black', size=15)
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)), 'P-value={:.2f}'.format(p_value), color='black', size=15)
    if old_elder1==6 and old_elder0==24:
        plt.savefig('C:/data/CORBIN/Graber/new/viz/adult_old/{}.png'.format(uge))
    elif old_elder1==6 and old_elder0==28:
        plt.savefig('C:/data/CORBIN/Graber/new/viz/adult_elder/{}.png'.format(uge))
    elif old_elder1==24 and old_elder0==28:
        plt.savefig('C:/data/CORBIN/Graber/new/viz/old_elder/{}.png'.format(uge))
    return slope, intercept, r**2, p_value

In [ ]:
df_ori = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older_f.xlsx', sheet_name='original')
df_end = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older_f.xlsx', sheet_name='end')

In [ ]:
df_ori['age'].value_counts().index[0]


In [ ]:
df_slope=[]
df_pval=[]
df_r2=[]
df_intercept=[]

for i in df_end['gene_id']:
    s, i, r2, p=regviz(df_ori, i)
    df_slope.append(s)
    df_pval.append(p)
    df_intercept.append(i)
    df_r2.append(r2)
df_end['slope']=df_slope
df_end['intercept']=df_intercept
df_end['R2']=df_r2
df_end['pval']=df_pval

In [ ]:
df_end.to_excel('C:/data/CORBIN/Graber/new/older_var.xlsx')

In [ ]:
df1_ori = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder_f.xlsx', sheet_name='original')
df1_end = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder_f.xlsx', sheet_name='end')

In [ ]:
df_slope=[]
df_pval=[]
df_r2=[]
df_intercept=[]

for i in df1_end['gene_id']:
    s, i, r2, p=regviz(df1_ori, i)
    df_slope.append(s)
    df_pval.append(p)
    df_intercept.append(i)
    df_r2.append(r2)
df1_end['slope']=df_slope
df1_end['intercept']=df_intercept
df1_end['R2']=df_r2
df1_end['pval']=df_pval

In [ ]:
df1_end.to_excel('C:/data/CORBIN/Graber/new/elder_var.xlsx')

In [ ]:
df_all = pd.read_excel('C:/data/CORBIN/Graber/new/all.xlsx', sheet_name='all')
dfa_older = pd.read_excel('C:/data/CORBIN/Graber/new/all.xlsx', sheet_name='older')
dfa_elder = pd.read_excel('C:/data/CORBIN/Graber/new/all.xlsx', sheet_name='elder')

In [ ]:
older_elder=pd.merge(dfa_older, dfa_elder, on='gene_id', how='outer')
dfa=pd.merge(older_elder, df_all, on='gene_id', how='left')
dfa.describe()

In [ ]:
dfa.to_excel('C:/data/CORBIN/Graber/new/all_1.xlsx')

In [ ]:
dfa_ori = pd.read_excel('C:/data/CORBIN/Graber/new/all_f.xlsx', sheet_name='original')
dfa_end = pd.read_excel('C:/data/CORBIN/Graber/new/all_f.xlsx', sheet_name='end')

In [ ]:
dfa_ori['age'].value_counts().index[1]

In [ ]:
df_slope=[]
df_pval=[]
df_r2=[]
df_intercept=[]

for i in dfa_end['gene_id']:
    s, i, r2, p=regviz(dfa_ori, i)
    df_slope.append(s)
    df_pval.append(p)
    df_intercept.append(i)
    df_r2.append(r2)
dfa_end['slope']=df_slope
dfa_end['intercept']=df_intercept
dfa_end['R2']=df_r2
dfa_end['pval']=df_pval

In [ ]:
dfa_end.to_excel('C:/data/CORBIN/Graber/new/all_var.xlsx')

In [ ]:
def regviz_all (data, uge):
    x=data['CFAB']
    y=data[uge]
    data_fit = np.polyfit(x, y, 1)
    slope, intercept, r, p_value, std_err = stats.linregress(x,y)
    fig, ax = plt.subplots(figsize=(10,8))
    
    #sns.regplot(x=x, y=y, color='deeppink', ci=0, marker='^')
    data1=data[data['age']==data['age'].value_counts().index[1]]
    data2=data[data['age']==data['age'].value_counts().index[0]]
    data3=data[data['age']==data['age'].value_counts().index[2]]
    ax.scatter(data1['CFAB'], data1[uge], c='black', marker='^', alpha=1, s=300)
    ax.scatter(data2['CFAB'], data2[uge], c='red', marker='^', alpha=1, s=300)
    ax.scatter(data3['CFAB'], data3[uge], c='green', marker='^', alpha=1, s=300)
#     ax1 = data1.plot(kind='scatter', x='cfab', y=uge, color='green', alpha=1, figsize=(10, 7), marker='^', s=400)
#     ax1 = data2.plot(kind='scatter', x='cfab', y=uge, color='red', alpha=1, figsize=(10, 7), marker='^', s=400)
    plt.plot(x, data_fit[0] * x + data_fit[1], color='darkblue', linestyle='dotted')
    old_elder0=data['age'].value_counts().index[0]
    old_elder1=data['age'].value_counts().index[1]
    old_elder2=data['age'].value_counts().index[2]
    plt.title('CFAB vs. '+uge+'(green:age {}, red:age {}, black:age {})'.format(old_elder2, old_elder0, old_elder1), size=24)
    
    plt.xlabel('CFAB', size=18)
    plt.ylabel('Log2 fold expression', size=18)
    if slope<0:
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/len(y), 'y={:.2f}+{:.2f}*x'.format(data_fit[1], data_fit[0]), color='black', size=15)
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/(len(y)-12), 'R2={:.2f}'.format(r**2), color='black', size=15)
        plt.text(max(x)-4, max(y)-(max(y)-min(y))/(len(y)-15), 'P-value={:.2f}'.format(p_value), color='black', size=15)
    else:
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)-15), 'y={:.2f}+{:.2f}*x'.format(data_fit[1], data_fit[0]), color='black', size=15)
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)-12), 'R2={:.2f}'.format(r**2), color='black', size=15)
        plt.text(max(x)-3, min(y)+(max(y)-min(y))/(len(y)), 'P-value={:.2f}'.format(p_value), color='black', size=15)
    
    plt.savefig('C:/data/CORBIN/Graber/new/viz/adult_old_elder/{}.png'.format(uge))

    return slope, intercept, r**2, p_value

In [ ]:
s, i, r2, p=regviz_all(dfc_ori, 'Gm2694')

In [ ]:
dfc_ori = pd.read_excel('C:/data/CORBIN/Graber/new/all_f.xlsx', sheet_name='original')
dfc_end = pd.read_excel('C:/data/CORBIN/Graber/new/all_f.xlsx', sheet_name='end')

In [ ]:
df_slope=[]
df_pval=[]
df_r2=[]
df_intercept=[]

for i in dfc_end['gene_id']:
    s, i, r2, p=regviz_all(dfc_ori, i)
    df_slope.append(s)
    df_pval.append(p)
    df_intercept.append(i)
    df_r2.append(r2)
dfc_end['slope']=df_slope
dfc_end['intercept']=df_intercept
dfc_end['R2']=df_r2
dfc_end['pval']=df_pval

In [ ]:
dfc_end.to_excel('C:/data/CORBIN/Graber/new/adult_old_elder_result.xlsx')

In [ ]:
dfo_cluster = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older_raw1.xlsx', index=False)



In [ ]:
age = dfo_cluster.pop("age")

lut = dict(zip(age.unique(), "rbg"))
row_colors = age.map(lut)
g = sns.clustermap(dfo_cluster, figsize=(12, 15),cmap="mako", row_colors=row_colors)

In [ ]:
dfe_cluster = pd.read_excel('C:/data/CORBIN/Graber/new/adult_elder_raw1.xlsx')


In [ ]:
age = dfe_cluster.pop("age")

lut = dict(zip(age.unique(), "rbg"))
row_colors = age.map(lut)
g = sns.clustermap(dfe_cluster, figsize=(12, 15),cmap="mako", row_colors=row_colors)

In [ ]:
dfa_cluster = pd.read_excel('C:/data/CORBIN/Graber/new/adult_older_elder_raw1.xlsx')


In [ ]:
age = dfa_cluster.pop("age")

lut = dict(zip(age.unique(), "rbg"))
row_colors = age.map(lut)
g = sns.clustermap(dfa_cluster, figsize=(12, 15),cmap="mako", row_colors=row_colors)

In [ ]:
dfexp=pd.read_excel('C:/data/CORBIN/Graber/new/all_1.xlsx', sheet_name='mean')

In [ ]:
dfexp_hm=dfexp.drop(['gene_id'],axis=1)
for i in dfexp_hm.columns:
    for j in range(len(dfexp_hm)):
        if abs(dfexp_hm[i][j])<=1.5:
            dfexp_hm[i][j]=np.nan
            
dfexp_hm

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors

In [ ]:
fig, ax = plt.subplots(figsize=(12,120)) 

#rdgn = sns.diverging_palette(h_neg=240, h_pos=10, s=98, l=65, sep=1,as_cmap=True)
cmap = LinearSegmentedColormap.from_list('GkR',['green','black','red'])
sns.heatmap(dfexp_hm, yticklabels=dfexp['gene_id'], cmap=cmap, cbar=True)

In [ ]:
dfr_fao = pd.read_excel('C:/data/CORBIN/Graber/new/lg2fc_ao.xlsx')
dfr_fae = pd.read_excel('C:/data/CORBIN/Graber/new/lg2fc_ae.xlsx')
dfr_ao=pd.read_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_old_result.xlsx')
dfr_ae=pd.read_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_elder_result.xlsx')
dfr_aoe=pd.read_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_old_elder_result.xlsx')
dfr_oe=pd.read_excel('C:/data/CORBIN/Graber/Graber_results0622/old_elderly_result.xlsx')



In [ ]:
dfr_all=pd.merge(dfr_fao, dfr_fae, on='gene_id', how='outer')

In [ ]:
dfr_ao=pd.merge(dfr_ao, dfr_fao, on='gene_id', how='left')
dfr_ae=pd.merge(dfr_ae, dfr_fae, on='gene_id', how='left')
dfr_aoe=pd.merge(dfr_aoe, dfr_all, on='gene_id', how='left')
dfr_oe=pd.merge(dfr_oe, dfr_all, on='gene_id', how='left')

In [ ]:
dfr_ao.to_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_older_reg.xlsx')
dfr_ae.to_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_elderly_reg.xlsx')
dfr_aoe.to_excel('C:/data/CORBIN/Graber/Graber_results0622/adult_older_elderly_reg.xlsx')
dfr_oe.to_excel('C:/data/CORBIN/Graber/Graber_results0622/older_elderly_reg.xlsx')

In [ ]:
df_go = pd.read_excel('C:/data/CORBIN/Graber/new/GOrilla_function_adult_older.xlsx', sheet_name='clean')

In [ ]:
sample=df_go['Genes'][2]
sample

In [ ]:
sample1=sample.split(',')
sample1[10]

In [ ]:
sample2=str(sample1[10]).split('-') 
sample2

In [ ]:
sample3=str(sample2[0])[1:-2]
sample3

In [ ]:
sample3[0].istitle()

In [ ]:
genes=[]
genes_all=[]
genes_go=[]
gene_term=[]
for i, row in df_go.iterrows():
    gene_len=len(df_go['Genes'][i].split(','))
    for j in range(gene_len):
        g=str(str(df_go['Genes'][i].split(',')[j]).split('-')[0])[1:-2]
        #print (g[0])
        if len(g)>1:
            if g[0].istitle():
                genes.append(g)
                genes_go.append(g)
    for k in range(len(genes)):
        gene_term.append(df_go['GO Term'][i])
    print (genes)
    genes_all.append(genes)
    genes=[]
    gene_len=0


In [ ]:
print (len(gene_term), gene_term)
print (len(genes_go), genes_go)

In [ ]:
data = {'GO_TERM': gene_term,
        'gene_id': genes_go}

df_older_gene = pd.DataFrame (data, columns = ['GO_TERM','gene_id'])
df_older_gene.to_excel('C:/data/CORBIN/Graber/new713/GOrilla_geneID_older.xlsx')

In [ ]:
genes_go_set=set(genes_go)
print (len(genes_go))
print (len(genes_go_set))
print (genes_go_set)

In [ ]:
df_go['gene_id']=genes_all
df_go.head()

In [ ]:
df_go.to_excel('C:/data/CORBIN/Graber/new/GOrilla_adult_older_cleaned.xlsx')

In [ ]:
df2_go = pd.read_excel('C:/data/CORBIN/Graber/new/GOrilla_function_adult_elderly.xlsx')

In [ ]:
genes=[]
genes_all=[]
genes_go2=[]
gene_term2=[]
for i, row in df2_go.iterrows():
    gene_len=len(df2_go['Genes'][i].split(','))
    for j in range(gene_len):
        g=str(str(df2_go['Genes'][i].split(',')[j]).split('-')[0])[1:-2]
        #print (g[0])
        if len(g)>1:
            if g[0].istitle():
                genes.append(g)
                genes_go2.append(g)
    for k in range(len(genes)):
        gene_term2.append(df2_go['GO Term'][i])
    print (genes)
    genes_all.append(genes)
    genes=[]
    gene_len=0


In [ ]:
print (len(gene_term2), gene_term2)
print (len(genes_go2), genes_go2)

In [ ]:
data = {'GO_TERM': gene_term2,
        'gene_id': genes_go2}

df_elderly_gene = pd.DataFrame (data, columns = ['GO_TERM','gene_id'])
df_elderly_gene.to_excel('C:/data/CORBIN/Graber/new713/GOrilla_geneID_elderly.xlsx')

In [ ]:
genes_go_set2=set(genes_go2)
print (len(genes_go2))
print (len(genes_go_set2))
print (genes_go_set2)

In [ ]:
df2_go['gene_id']=genes_all
df2_go.head()

In [ ]:
df2_go.to_excel('C:/data/CORBIN/Graber/new/GOrilla_adult_elderly_cleaned.xlsx')